# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察する。これにより、特別な学習なしでのモデルの限界を確認する。

2. **作成したテキストの活用**  
   リサーチを行い作成したテキストをモデルに学習させ、RAGを導入する。


## 扱う質問

「NintetndoSwich2」に関する質問を取り扱う

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用する。このモデルは、リリース時期の関係上、「Switch2」が発表される前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [ ]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [ ]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
questions = [
    "Nintendo Switch2の国内専用本体価格はいくらですか",
    "Nintendo Switchの本体価格はいくらですか",
    "Nintendo Switch2の発売日はいつですか",
    "Nintendo SwitchとNintendo Switch2の違いを教えてください",
    "Nintendo Switchの発売日はいつですか"
]

for question in questions:
    response = generate_output(question)
    print(f"質問: {question}")
    print(f"回答: {response}\n")

## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「Nintendo Switch2」についての情報を知らない旨の回答をし、ハルシネーションは起こさなかった。モデルの作成時期の関係で、Switch2の情報を持たないと考えられる。

---

# 2. 作成したテキストの活用

## 2.1 作成したテキストをソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入する：

* **知識ソース**: 自身でリサーチして作成したテキスト
* **目的**: モデルに「NintendoSwitch2」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**:作成したテキストを使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

In [ ]:
with open("/about_switch2.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[40])

In [ ]:
# Retrievalの実行
questions = [
    "Nintendo Switch2の国内専用本体価格はいくらですか",
    "Nintendo Switchの本体価格はいくらですか",
    "Nintendo Switch2の発売日はいつですか",
    "Nintendo SwitchとNintendo Switch2の違いを教えてください",
    "Nintendo Switchの発売日はいつですか"
]

for question in questions:
    query_embeddings = emb_model.encode([question], prompt_name="query")
    document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
questions = [
    "Nintendo Switch2の国内専用本体価格はいくらですか",
    "Nintendo Switchの本体価格はいくらですか",
    "Nintendo Switch2の発売日はいつですか",
    "Nintendo SwitchとNintendo Switch2の違いを教えてください",
    "Nintendo Switchの発売日はいつですか"
]

for question in questions:
    query_embeddings = emb_model.encode([question], prompt_name="query")
    document_embeddings = emb_model.encode(documents)
    references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
    query =  f"[参考資料]\n{references}\n\n[質問]"+question
    response = generate_output(query)
    print(f"質問: {question}")
    print(f"回答: {response}\n")

### 結果 (初期RAG実装)

RAG導入後はモデル作成時以降のトピックについての説明を、参考資料をもとにして正確にすることができていた。しかし、RAG実装前に回答出来ていた質問「Nintendo Switchの本体価格はいくらですか」に対して、「Nintendo Switchの本体価格は **49980円（税込）** です。」という回答が見られた。これは、 Switch2の本体価格であり、モデルが元々持っていた知識を参照せず、参考資料にある知識を参照した、さらに、Switch2と Switchという似た単語の区別をモデルができなかったことで起きたと考えられる。解決法としては、モデルにSwitch2と Switchが別のものであることを念入りに学習させることなどが挙げられる。